In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
# Collect 20% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_20, _ , y_20 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.2, stratify=y_label_0_1, random_state=42)


In [13]:
X_20.shape

(182354, 56)

In [6]:
from xgboost import XGBClassifier

print('xgb')
xgb20 = XGBClassifier()
xgb20.fit(X_20, y_20)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf20 = RandomForestClassifier()
rf20.fit(X_20, y_20)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt20 = DecisionTreeClassifier()
dt20.fit(X_20, y_20)

xgb
RF
DT


DecisionTreeClassifier()

In [7]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb20.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ExcludeCaFA/UNSW_Input20/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ExcludeCaFA/UNSW_Input20/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [  2522    193   2313    910     57    924    149 106712    120     71]
(2522, 56) (2522,)
Save 0 to baseline_Def1.npy
(193, 56) (193,)
Save 1 to baseline_Def2.npy
(2313, 56) (2313,)
Save 2 to baseline_Def3.npy
(910, 56) (910,)
Save 3 to baseline_Def4.npy
(57, 56) (57,)
Save 4 to baseline_Def5.npy
(924, 56) (924,)
Save 5 to baseline_Def6.npy
(149, 56) (149,)
Save 7 to baseline_Def8.npy
(106712, 56) (106712,)
Save 8 to baseline_Def9.npy
(120, 56) (120,)
Save 9 to baseline_Def10.npy
(71, 56) (71,)
Save 10 to baseline_Def11.npy
Execution Time: 0.273994 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8 10] [100838     65   1083    652     99 181069      4     58  57966     79]
(100838, 56) (100838,)
Save 0 to BIM_Def1.npy
(65, 56) (65,)
Save 1 to BIM_Def2.npy
(1083, 56) (1083,)
Save 2 to BIM_Def3.npy
(652, 56) (652,)
Save 3 to BIM_Def4.npy
(99, 56) (99,)
Save 4 to BIM_Def5.npy
(181069, 56) (181069,)
Save 5 to BIM_Def6.npy
(4, 56) (4,)
Save 6 to BIM_Def

In [8]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf20.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input20/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input20/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [  2980    160   2268    918     66   1307    163 105946    104     59]
(2980, 56) (2980,)
Save 0 to baseline_Def1.npy
(160, 56) (160,)
Save 1 to baseline_Def2.npy
(2268, 56) (2268,)
Save 2 to baseline_Def3.npy
(918, 56) (918,)
Save 3 to baseline_Def4.npy
(66, 56) (66,)
Save 4 to baseline_Def5.npy
(1307, 56) (1307,)
Save 5 to baseline_Def6.npy
(163, 56) (163,)
Save 7 to baseline_Def8.npy
(105946, 56) (105946,)
Save 8 to baseline_Def9.npy
(104, 56) (104,)
Save 9 to baseline_Def10.npy
(59, 56) (59,)
Save 10 to baseline_Def11.npy
Execution Time: 1.101441 seconds
(341913,)
[0 2 3 4 5 7 8] [119896    383    487      2 192485     24  28636]
(119896, 56) (119896,)
Save 0 to BIM_Def1.npy
(383, 56) (383,)
Save 2 to BIM_Def3.npy
(487, 56) (487,)
Save 3 to BIM_Def4.npy
(2, 56) (2,)
Save 4 to BIM_Def5.npy
(192485, 56) (192485,)
Save 5 to BIM_Def6.npy
(24, 56) (24,)
Save 7 to BIM_Def8.npy
(28636, 56) (28636,)
Save 8 to BIM_Def9.npy
Execution Time: 2.623479 

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt20.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ExcludeCaFA/UNSW_Input20/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ExcludeCaFA/UNSW_Input20/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [  4544    441   2541   1946    195   1970    433 101478    218    205]
(4544, 56) (4544,)
Save 0 to baseline_Def1.npy
(441, 56) (441,)
Save 1 to baseline_Def2.npy
(2541, 56) (2541,)
Save 2 to baseline_Def3.npy
(1946, 56) (1946,)
Save 3 to baseline_Def4.npy
(195, 56) (195,)
Save 4 to baseline_Def5.npy
(1970, 56) (1970,)
Save 5 to baseline_Def6.npy
(433, 56) (433,)
Save 7 to baseline_Def8.npy
(101478, 56) (101478,)
Save 8 to baseline_Def9.npy
(218, 56) (218,)
Save 9 to baseline_Def10.npy
(205, 56) (205,)
Save 10 to baseline_Def11.npy
Execution Time: 0.077518 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [123302   1564   8930   9051   1408 149532     35  10334  37287    123
    347]
(123302, 56) (123302,)
Save 0 to BIM_Def1.npy
(1564, 56) (1564,)
Save 1 to BIM_Def2.npy
(8930, 56) (8930,)
Save 2 to BIM_Def3.npy
(9051, 56) (9051,)
Save 3 to BIM_Def4.npy
(1408, 56) (1408,)
Save 4 to BIM_Def5.npy
(149532, 56) (149532,)
Save 5 to BIM_Def6.npy
(

In [10]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}20")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_ExcludeCaFA/UNSW_Input20/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [11]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input20_ExcludeCaFA.csv")

In [12]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB20,2522,193,2313,910,57,924,0,149,106712,120,71,113971
1,XGB20,100838,65,1083,652,99,181069,4,58,57966,0,79,341913
2,XGB20,99187,58,1010,604,126,183610,73,712,56517,0,16,341913
3,XGB20,100838,65,1083,652,99,181069,4,58,57966,0,79,341913
4,XGB20,74637,487,2104,350,82,244616,374,17505,1402,291,65,341913
5,XGB20,105750,441,4431,2477,198,177560,12,358,50241,266,179,341913
6,XGB20,6819,554,6807,2727,166,2635,0,447,321225,319,214,341913
7,XGB20,89596,19,1455,1252,15,2180,0,73,247265,15,43,341913
8,XGB20,101163,83,536,757,44,176235,2,33,63034,1,25,341913
9,XGB20,99744,64,1144,632,211,178211,0,128,61700,2,77,341913
